## Session 14 Assignment 1

In this assignment students need to predict whether a person makes over 50K per year
or not from classic adult dataset using XGBoost. The description of the dataset is as
follows:

Data Set Information:

Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

In [19]:
# Importing module and assigning alias for them.
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

Attribute Information:

Listing of attributes:

>50K, <=50K.

age: continuous.

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov,
Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc,
9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France,
Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [20]:
# loading a taining and test dataset from URL into a pandas dataframe.
col_name = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','avg-hours-per-week','native-country','income']
train_set=pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', names = col_name)
test_set = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test", names = col_name)

In [21]:
print("Training Data-set:\n")
train_set.head()

Training Data-set:



,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,avg-hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [22]:
print("Testing Data-set:\n")
test_set.head()

Testing Data-set:



,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,avg-hours-per-week,native-country,income
0,|1x3 Cross validator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.


In [23]:
# removing first row from test data set as it is irrelevant
test_set.drop(test_set.index[[0]],inplace=True)

In [24]:
print("Testing Data-set:\n")
test_set.head()

Testing Data-set:



,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,avg-hours-per-week,native-country,income
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.
5,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K.


In [25]:
# Converting "income" column '>50K' as 1 and '<=50K' as 0
train_set["income"] = train_set["income"].map({ " <=50K": 0, " >50K": 1 })
test_set["income"] = test_set["income"].map({ " <=50K.": 0, " >50K.": 1 })

In [26]:
print("Training Data-set:\n")
train_set.head()

Training Data-set:



,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,avg-hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [27]:
print("Testing Data-set:\n")
test_set.head()

Testing Data-set:



,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,avg-hours-per-week,native-country,income
1,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,0
2,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,0
3,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,1
4,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,1
5,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,0


In [28]:
# Encoding all categorical column of "train_set" and "test_set" data
for x in train_set.columns:
    if train_set[x].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_set[x].values))
        train_set[x] = lbl.transform(list(train_set[x].values))
for x in test_set.columns:
    if test_set[x].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(test_set[x].values))
        test_set[x] = lbl.transform(list(test_set[x].values))

In [29]:
# Splitting the dataset into 'X' and 'Y'column for Training set and Test set
X_train= train_set.iloc[:, :14].values
y_train= train_set.iloc[:, 14].values
X_test= test_set.iloc[:, :14].values
y_test= test_set.iloc[:, 14].values
print("X-TRAINING DATA:\n",X_train)
print("X-TEST DATA:\n",X_test)
print("Y-TRAINING DATA:\n",y_train)
print("Y-TEST DATA:\n",y_test)

X-TRAINING DATA:
 [[    39      7  77516 ...      0     40     39]
 [    50      6  83311 ...      0     13     39]
 [    38      4 215646 ...      0     40     39]
 ...
 [    58      4 151910 ...      0     40     39]
 [    22      4 201490 ...      0     20     39]
 [    52      5 287927 ...      0     40     39]]
X-TEST DATA:
 [[8.00000e+00 4.00000e+00 2.26802e+05 ... 0.00000e+00 4.00000e+01
  3.80000e+01]
 [2.10000e+01 4.00000e+00 8.98140e+04 ... 0.00000e+00 5.00000e+01
  3.80000e+01]
 [1.10000e+01 2.00000e+00 3.36951e+05 ... 0.00000e+00 4.00000e+01
  3.80000e+01]
 ...
 [2.10000e+01 4.00000e+00 3.74983e+05 ... 0.00000e+00 5.00000e+01
  3.80000e+01]
 [2.70000e+01 4.00000e+00 8.38910e+04 ... 0.00000e+00 4.00000e+01
  3.80000e+01]
 [1.80000e+01 5.00000e+00 1.82148e+05 ... 0.00000e+00 6.00000e+01
  3.80000e+01]]
Y-TRAINING DATA:
 [0 0 0 ... 0 0 1]
Y-TEST DATA:
 [0 0 1 ... 0 0 1]


In [33]:
# Feature Scaling X data set

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print("X-TRAINING DATA AFTER FEATURE SCALING:\n",X_train)
print("X-TEST DATA AFTER FEATURE SCALING:\n",X_test)

X-TRAINING DATA AFTER FEATURE SCALING:
 [[ 0.03067056  2.15057856 -1.06361075 ... -0.21665953 -0.03542945
   0.29156857]
 [ 0.83710898  1.46373585 -1.008707   ... -0.21665953 -2.22215312
   0.29156857]
 [-0.04264203  0.09005041  0.2450785  ... -0.21665953 -0.03542945
   0.29156857]
 ...
 [ 1.42360965  0.09005041 -0.35877741 ... -0.21665953 -0.03542945
   0.29156857]
 [-1.21564337  0.09005041  0.11095988 ... -0.21665953 -1.65522476
   0.29156857]
 [ 0.98373415  0.77689313  0.92989258 ... -0.21665953 -0.03542945
   0.29156857]]
X-TEST DATA AFTER FEATURE SCALING:
 [[-2.24201954  0.09005041  0.35077413 ... -0.21665953 -0.03542945
   0.16375118]
 [-1.28895595  0.09005041 -0.94709543 ... -0.21665953  0.77446821
   0.16375118]
 [-2.02208179 -1.28363502  1.39436215 ... -0.21665953 -0.03542945
   0.16375118]
 ...
 [-1.28895595  0.09005041  1.75468988 ... -0.21665953  0.77446821
   0.16375118]
 [-0.84908045  0.09005041 -1.00321189 ... -0.21665953 -0.03542945
   0.16375118]
 [-1.5088937   0.77689

In [34]:
# Fitting XGBoost classifier to the Training set
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [35]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
print(y_pred)

[0 0 0 ... 0 0 0]


In [36]:
cm = confusion_matrix(y_test, y_pred)
print("CONFUSIONMATRIX:\n",cm)

CONFUSIONMATRIX:
 [[12239   196]
 [ 2409  1437]]


In [37]:
# Accuracy Prediction
count_misclassified = (y_test != y_pred).sum()
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = metrics.accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))

Misclassified samples: 2605
Accuracy: 0.84


In [38]:
#Classification Report for XG-Boost Classifier model
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.98      0.90     12435
           1       0.88      0.37      0.52      3846

   micro avg       0.84      0.84      0.84     16281
   macro avg       0.86      0.68      0.71     16281
weighted avg       0.85      0.84      0.81     16281



In [39]:
# Log loss for XG-Boost Classifier model
metrics.log_loss(y_test,y_pred)

5.526298705843538